# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{bedrock-webinar-July-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave requests openai
```


In [114]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
import openai
from enum import Enum
from pydantic import BaseModel, Field
from rich.panel import Panel
from rich.markdown import Markdown
from rich.console import Console as RichConsole
from exa_py import Exa
from typing import Any, Callable, Dict, List, get_type_hints

Define a model to use, as we are going to use tool calling you need a capable model like `mistral-large`

In [115]:
class Console(RichConsole):
    def md(self, text): 
        return self.print(Markdown(text))

console = Console()

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "zai-org/GLM-4.5"
MODEL_LARGE = "moonshotai/Kimi-K2-Instruct"

oai_client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project="milieu/london-workshop-2025")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [6]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init('milieu/london-workshop-2025')

/Users/tcapelle/work/deep-research-bot/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: Logged in as Weights & Biases user: capecape.
weave: View Weave data at https://wandb.ai/milieu/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](images/01_trace.png)

In [7]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
)
# Print the response content
assistant_response = response.choices[0].message.content
console.md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed19-ebbf-7b1e-88de-b63eb64a6f40


LLM Response:\nHello! Great question! 😊                                                                           

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "assistant" that can operate autonomously or semi-autonomously.         

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or inputs. This could be:                         

 • Text input (like your message to me)                                                                            
 • Sensor data (in robotics)                                                                                       
 • Camera images (in self-driving cars)                                                                            
 • Web APIs, databases, etc.                                                                                       

👉 Example: A chatbot "perceives" your message as text input.                                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     2. Decision-Making (Reasoning & Planning)                                     

Using the input, the agent processes the information and decides what to do. This involves:                        

 • Understanding the input (e.g., natural language processing)                                                     
 • Reasoning (e.g., logic, inference, or machine learning models)                                                  
 • Planning a sequence of actions to achieve a goal                                                                

👉 Example: I analyze your question, understand you're asking about AI agents, and decide how to structure a       
helpful answer.                                                                                                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action based on its decision. Actions can include:                                           

 • Generating a response (like this one!)                                                                          
 • Moving a robot arm                                                                                              
 • Recommending a product                                                                                          
 • Playing a move in a game                                                                                        

👉 Example: I type out this explanation and send it back to you.                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        4. Learning (Optional but Powerful)                                        

Many modern AI agents can learn from experience using techniques like:                                             

 • Reinforcement learning (learning by trial and error)                                                            
 • Supervised learning (learning from labeled examples)                                                            
 • Fine-tuning on new data                                                                                         

👉 Example: A recommendation agent learns w

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed2a-a602-7e22-bebf-cd3b4cea31cf


Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [8]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
console.md(response.content)

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-3814-723c-9370-31e4f081fa70


Hello! Great question! 😊 An AI agent is a software system that perceives its environment, makes decisions, and    
takes actions to achieve specific goals. Think of it like a digital "entity" that can act autonomously to perform  
tasks on your behalf.                                                                                              

Here’s a simple breakdown of how an AI agent works:                                                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or inputs. This could include:                    

 • Text input (like your message to me)                                                                            
 • Sensor data (in robotics)                                                                                       
 • Camera images (in self-driving cars)                                                                            
 • Database queries                                                                                                
 • Web scraping                                                                                                    

▌ Example: When you type a question, I (as an AI agent) perceive it as text input.                               

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     2. Decision-Making (Reasoning & Planning)                                     

Using the input, the agent processes the information and decides what to do. This involves:                        

 • Understanding the goal (e.g., answer correctly, win a game, control a robot)                                    
 • Reasoning (applying logic, knowledge, or learned patterns)                                                      
 • Planning (figuring out a sequence of steps)                                                                     
 • Sometimes using machine learning models (like neural networks) to predict or classify                           

▌ Example: I analyze your question, understand the intent, retrieve relevant knowledge, and plan a helpful       
▌ response.                                                                                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action using actuators or output mechanisms:                                                 

 • Generating text (like this response)                                                                            
 • Moving a robot arm                                                                                              
 • Playing a move in a game                                                                                        
 • Sending a command to another system                                                                             

▌ Example: I send this message back to you.                                                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        4. Learning (Optional but Powerful)                                        

Many modern AI agents can learn from experience:                                                                   

 • Reinforcement learning (learning by trial and error)                                                            
 • Updating models based on feedback             

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-635c-722c-b2da-9d8f9113748c
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-6362-7c61-9fee-58c27c5cd484
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-8508-79f1-8bb9-bccbbcd8581e
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1b-2c37-73e6-b03c-972e154d043c
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1b-329c-790f-ba3b-b66a860281d4
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed20-20f7-7c85-98f1-173abde134d4
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed22-6920-71a3-b7bd-b7c51aca87c7
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed22-f15e-745b-a901-c123d20bb8f5
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed24-2f8e-7336-9259-ca9a91f1e340
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed28-2cca-7370-9c1d-2bb801f4ff15
weave: 🍩 https://wan

![](images/02_nested_trace.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](images/function-calling-diagram-steps.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [67]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Use this tool to add numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [68]:
add_numbers(1, 2)

3

In [11]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

TypeError: Object of type function is not JSON serializable

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [12]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [13]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-2f61cc755a7f4d948fbefe7337dafaaf',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1b-5273-7eb8-8809-279d0bb5a592
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1f-b6f6-7007-93ed-daf341da99b3
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed22-f6b2-7a86-8f47-f3bd686aac85
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed29-1826-7521-aa73-ca589c40dccd
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed31-a2a7-7340-b7b8-60183c8365a0


## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [14]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-90d7-72b8-987c-48799929d407


Final Result: 88

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1f-70f3-759f-ac25-e1851c53a332
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed20-2863-77d4-bfbc-2c55b52a644d
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed24-3478-711f-8fa0-88967c3eb155
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed3a-6fdd-71c9-a6be-eca1beda7764
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed41-190a-7fe5-a617-32b3b962dc2c
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed68-05b6-7e14-bb51-60ead2ad2094
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed68-ffa2-7644-88a5-5747d8def0d8


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [15]:
messages.append(response.model_dump())

In [16]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [17]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-2f61cc755a7f4d948fbefe7337dafaaf',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-2f61cc755a7f4d948fbefe7337dafaaf',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [18]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [ ]:
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-aa19-7719-9ea9-0f7c24a3b435


The sum of your lucky numbers, 77 and 11, is **88**.

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1b-5954-7fa6-8d74-f10aa9a671ff
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed29-126f-7186-bd8b-c09c4397c924
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed2c-56aa-7966-99f7-0c825e4bc2eb
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed35-54e5-76db-842d-630c6d3e2a71
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed38-2efa-7467-8d3b-43670925ae99
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed3b-9a9d-767c-a150-c3d32a284754


## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [20]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [21]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [22]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [71]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)


{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [24]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [25]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1a-cad4-799a-9e88-44665f159aef


3

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed32-0899-712b-83ff-95b1ca9753fc
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed37-e8b0-70b3-a402-0205fd7dee33
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed67-7996-7d00-90a8-13e269727f9f


### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [26]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [27]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.masterclass.com/articles/how-to-cook-sea-bass',
            id='https://www.masterclass.com/articles/how-to-cook-sea-bass',
            title='How to Cook Sea Bass: Simple Mediterranean Sea Bass Recipe - 2025 - MasterClass',
            score=None,
            published_date='2025-03-05T00:00:00.000Z',
            author='',
            image='',
            favicon=None,
            subpages=None,
            extras=None,
            text='Is MasterClass right for me?\n\nTake this quiz to find out.\n\nSign 
Up\n\n[All](https://www.masterclass.com/articles/categories/all-classes)[Community and 
Government](https://www.masterclass.com/articles/categories/community-and-government)[Wellness](https://www.masterc
lass.com/articles/categories/wellness)[Design & 
Style](https://www.masterclass.com/articles/categories/design-style)[Arts & 
Entertainment](https://www.masterclass.com/articles/categories/arts-entertainment)[Writing](https://www.masterclass
.com/articles/categories/writing)[Sports & 
Gaming](https://www.masterclass.com/articles/categories/sports-gaming)[Science & 
Tech](https://www.masterclass.com/articles/categories/science-tech)[Music](https://www.masterclass.com/articles/cat
egories/music)[Food](https://www.masterclass.com/articles/categories/food)[Home & 
Lifestyle](https://www.masterclass.com/articles/categories/home-lifestyle)[Business](https://www.masterclass.com/ar
ticles/categories/business)\n\n[Food](https://www.masterclass.com/articles/categories/food)\n\n# How to Cook Sea 
Bass: Simple Mediterranean Sea Bass Recipe\n\nWritten by MasterClass\n\nLast updated: Mar 5, 2025 • 1 min 
read\n\nMediterranean sea bass is an easy-to-cook fish that’s simple enough for a weeknight and luxurious enough 
for a special occasion.\n\n## Learn From the Best\n\nTeaches Gardening\n\nTeaches Intentional Eating\n\nThe 
Greatest of All Time\n\nGut Health with Leading Experts\n\nBrain Health with Leading Experts\n\nTeaches Cooking 
I\n\nTeaches Cooking\n\nTeaches the Art of Home Cooking\n\nTeaches Cooking Techniques I: Vegetables, Pasta, and 
Eggs\n\nTeaches Cooking II: Restaurant Recipes at Home\n\nTeaches Cooking Techniques II: Meats, Stocks, and 
Sauces\n\nTeaches French Pastry Fundamentals\n\nTeaches Wine Appreciation\n\nTeaches Texas-Style BBQ\n\nTeaches 
Modern Italian Cooking\n\nTeaches Cooking Techniques III: Seafood, Sous Vide, and Desserts\n\nTeaches Mexican 
Cooking\n\nTeach Mixology\n\nTeaches Modern Middle Eastern Cooking\n\nTeaches Bread Baking\n\nTeaches Modern 
Japanese Cooking\n\nTeaches Southern Cooking\n\nTeaches Indian Cooking\n\nTeaches Intuitive Cooking\n\nTeaches 
Tracing Your Roots Through Food\n\nTeaches Modern Vegetarian Cooking\n\nSohla El-Waylly with Special Guest 
Chefs\n\nJosé Andrés with Special Guest Chefs\n\nWith Emily Wines\n\nTeaches Gardening\n\nTeaches Intentional 
Eating\n\nThe Greatest of All Time\n\nGut Health with Leading Experts\n\nBrain Health with Leading 
Experts\n\nTeaches Cooking I\n\nTeaches Cooking\n\nTeaches the Art of Home Cooking\n\nTeaches Cooking Techniques I:
Vegetables, Pasta, and Eggs\n\nTeaches Cooking II: Restaurant Recipes at Home\n\nTeaches Cooking Techniques II: 
Meats, Stocks, and Sauces\n\nTeaches French Pastry Fundamentals\n\nTeaches Wine Appreciation\n\nTeaches Texas-Style
BBQ\n\nTeaches Modern Italian Cooking\n\nTeaches Cooking Techniques III: Seafood, Sous Vide, and 
Desserts\n\nTeaches Mexican Cooking\n\nTeach Mixology\n\nTeaches Modern Middle Eastern Cooking\n\nTeaches Bread 
Baking\n\nTeaches Modern Japanese Cooking\n\nTeaches Southern Cooking\n\nTeaches Indian Cooking\n\nTeaches 
Intuitive Cooking\n\nTeaches Tracing Your Roots Through Food\n\nTeaches Modern Vegetarian Cooking\n\nSohla 
El-Waylly with Special Guest Chefs\n\nJosé Andrés with Special Guest Chefs\n\nWith Emily 
Wines\n\n[Get](https://www.masterclass.com/find-my-classes)\n\n## What to Serve With Mediterranean Sea Bass\n\nLike
mo

Let's explore the payload

In [28]:
console.md("\n-------------------\n".join(result.text for result in search_res.results))

Is MasterClass right for me?                                                                                       

Take this quiz to find out.                                                                                        

Sign Up                                                                                                            

]8;id=275357;https://www.masterclass.com/articles/categories/all-classes\All]8;;\]8;id=496098;https://www.masterclass.com/articles/categories/community-and-government\Community and Government]8;;\]8;id=297709;https://www.masterclass.com/articles/categories/wellness\Wellness]8;;\]8;id=908109;https://www.masterclass.com/articles/categories/design-style\Design & Style]8;;\]8;id=493282;https://www.masterclass.com/articles/categories/arts-entertainment\Arts & Entertainment]8;;\]8;id=746152;https://www.masterclass.com/articles/categories/writing\Writing]8;;\]8;id=589798;https://www.masterclass.com/articles/categories/sports-gaming\Sports & Gaming]8;;\]8;id=396967;https://www.masterclass.com/articles/categories/science-tech\Science & ]8;;\              
]8;id=396967;https://www.masterclass.com/articles/categories/science-tech\Tech]8;;\]8;id=165489;https://www.masterclass.com/articles/categories/music\Music]8;;\]8;id=262871;https://www.masterclass.com/articles/categories/food\Food]8;;\]8;id=811999;https://www.masterclass.com/articles/categories/home-lifestyle\Home & Lifestyle]8;;\]8;id=443021;https://www.masterclass.com/articles/categories/business\Business]8;;\                                                                              

]8;id=779049;https://www.masterclass.com/articles/categories/food\Food]8;;\                                                                                                               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                           How to Cook Sea Bass: Simple Mediterranean Sea Bass Recipe                            ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Written by MasterClass                                                                                             

Last updated: Mar 5, 2025 • 1 min read                                                                             

Mediterranean sea bass is an easy-to-cook fish that’s simple enough for a weeknight and luxurious enough for a     
special occasion.                                                                                                  


                                                Learn From the Best                                                

Teaches Gardening                                                                                                  

Teaches Intentional Eating                                                                                         

The Greatest of All Time                                                                                           

Gut Health with Leading Experts                                                                                    

Brain Health with Leading Experts                                                                                  

Teaches Cooking I                                                                                                  

Teaches Cooking                                                                                                    

Teaches the Art of Home Cooking                                                                                    

Teaches Cooking Techniques I: Vegetables, Pasta, and Eggs                                                          

Teaches Cooking II: Restaurant Recipes at Home                                                                     

Teaches Cooking Techniques II: Meats, Stocks, and

In [29]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output

    

In [30]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [31]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

[
    {
        'title': '空气炸锅烤鲈鱼_小视频',
        'text': '_\ue750_\n\n空气炸锅烤鲈鱼\\_小视频\n\n\ue66a\n\n[_@_ 
运营今天吃什么吖](https://m.baidu.com/from=0/bd_page_type=1/ssid=0/uid=0/pu=usm%400%2Csz%401320_220%2Cta%40qbase___
24_128.0/baiduid=6708B0310C29778AEB74C6BF35A4C80B/w=0_10_/t=iphone/l=1/tc?ref=www_iphone&lid=7319397126472011889&pd
=video_page&fm=alop&isAtom=1&clk_info=%7B%22applid%22%3A%227530424980198381335%22%2C%22apptpl%22%3A%22normal%22%2C%
22frsrcid%22%3A%224660%22%2C%22atn%22%3A%22verticalDemote%22%7D&is_baidu=0&applid=7530424980198381335&module=sf&wd=
&eqid=6881719f2073d7171000000668f04424&w_qd=IlPT2AEptyoA_yky-RobyxSvHVtOpoUpxV1MhwXRwuK&bdver=2_1&tcplug=1&dict=-1&
sec=7249&di=21d34ab30f0b41ef&bdenc=1&nsrc=bKGPsuucXsi16agaDipdDMnQ0C5PLkrR3YAy7bfdXrCsyki9Wej2XBAY7BMsPivCHwqc8nUFm
xi%2FkAxZhmxR%2BJe4wfAlXRaPSz9kY%2F5qFaA%3D)\n\n### 
空气炸锅烤鱼详细做法,肉质鲜嫩,老公一口气吃了3条\n\n[服务说明](https://m.baidu.com/from=0/bd_page_type=1/ssid=0/uid=
0/pu=usm%400%2Csz%401320_220%2Cta%40qbase___24_128.0/baiduid=6708B0310C29778AEB74C6BF35A4C80B/w=0_10_/t=iphone/l=1/
tc?ref=www_iphone&lid=7319397126472011889&pd=video_page&fm=alop&isAtom=1&clk_info=%7B%22applid%22%3A%22753042498019
8381335%22%2C%22apptpl%22%3A%22normal%22%2C%22frsrcid%22%3A%224660%22%2C%22atn%22%3A%22verticalDemote%22%7D&is_baid
u=0&applid=7530424980198381335&module=sf&wd=&eqid=6881719f2073d7171000000668f04424&w_qd=IlPT2AEptyoA_yky-RobyxSvHVt
OpoUpxV1MhwXRwuK&bdver=2_1&tcplug=1&dict=-1&sec=7249&di=23ae4357e8e103ae&bdenc=1&nsrc=5P19DeNEQ79MU%2BCTJ2pHCtVjAuy
RdRnuZWIqBgU8XEn8bu%2FZecAAgxBcitCPKQF2Yu3ijSOIzsJ6E2ztTqoe1HMkARRj%2BpSw83kmOfsm5i5o9DDobe2B2B%2BPNZeHgIES) 
[查看原网页](https://m.baidu.com/from=0/bd_page_type=1/ssid=0/uid=0/pu=usm%400%2Csz%401320_220%2Cta%40qbase___24_12
8.0/baiduid=6708B0310C29778AEB74C6BF35A4C80B/w=0_10_/t=iphone/l=1/tc?ref=www_iphone&lid=7319397126472011889&pd=vide
o_page&fm=alop&isAtom=1&clk_info=%7B%22applid%22%3A%227530424980198381335%22%2C%22apptpl%22%3A%22normal%22%2C%22frs
rcid%22%3A%224660%22%2C%22atn%22%3A%22verticalDemote%22%7D&is_baidu=0&applid=7530424980198381335&module=sf&wd=&eqid
=6881719f2073d7171000000668f04424&w_qd=IlPT2AEptyoA_yky-RobyxSvHVtOpoUpxV1MhwXRwuK&bdver=2_1&tcplug=1&dict=-1&sec=7
249&di=b1ef7d3001220bef&bdenc=1&nsrc=sV4Lyj3wDfldjZHLSHmPNGJDBM68yQFyGByx31Vohfh4ikmHjhm7hFqzF960Zf8RFcqZ3k92q3MzEY
8ph68fOkNoDEH1pPicchioh9drMdG62uU%2Bo2x9STraKPIMGQU0JNQu4WMJZ9ifTL5fMWWX76vQDptBvTTt%2BBIZOPQUjWs%3D)\n\n更多精彩内
容\n\n_\ue780_\n\n加载中...\n\n[假如我有一个空气炸锅 用空气炸锅做的烤鱼,做法简单,还好吃\\\n\\\n_\ue732_ 
1641](https://m.baidu.com/from=0/bd_page_type=1/ssid=0/uid=0/pu=usm%400%2Csz%401320_220%2Cta%40qbase___24_128.0/bai
duid=6708B0310C29778AEB74C6BF35A4C80B/w=0_10_/t=iphone/l=1/tc?ref=www_iphone&lid=7319397126472011889&pd=video_page&
fm=alop&cyc=1&isAtom=1&clk_info=%7B%22applid%22%3A%227530424980198381335%22%2C%22apptpl%22%3A%22normal%22%2C%22frsr
cid%22%3A%224660%22%2C%22atn%22%3A%22verticalDemote%22%7D&is_baidu=0&applid=7530424980198381335&module=sf&wd=&eqid=
6881719f2073d7171000000668f04424&w_qd=IlPT2AEptyoA_yky-RobyxSvHVtOpoUpxV1MhwXRwuK&bdver=2_1&tcplug=1&dict=-1&sec=72
49&di=60f7ae75defac2d7&bdenc=1&nsrc=mMQ82pJog0pERasoA4JVBDu4U7tqCFG5u9gVhkJAuswoIqcHSTPJPXEOO20YxGwpvhNe%2F3ShgJZTu
XLk0NLx38eoTrB9VOic9SMoCFskRmdlpVuNuBonh%2B%2FEgF0dM0yuE1zY%2FAvtA09IsdLL%2B288rFawnbeTO1VFWznBm5qPx2YwCRhCjwzbTvmU
jOM1ze3oysDDCJ4128BjAOp76fzMu%2FNZe0shUJFcS%2FNd98lrev%2BcZyr16adladIEbCv2xCaesQjFI6rNen9rsj9dEMKHx0%2B9U495eQHUgr%
2FtH%2Bkfa2PNlsw2Jjkl49%2FCDF1XbRlAghifchOeST8IdO0XgA5O0Pco094NFOhJ%2FJrcvIVd5k3dPBGTGyse3M7hVfx1Lp%2FgyTduaoISbGAv
qgQxImfYCyw0fbm08aQAuRsteomCkUDGNuMoAq7FY%2BZhAUQgSKpqU%2Fxj2mNE6o%2FwamyGqMCKKigHQ8M9UNK2oC8DcqCFgPvSAbcb6%2BFpuQq
O%2BoNtNwIml2tV2MgxbpfLOKINhIebNs54rw2e0tSmNNhu1KvTuD7lXIy6ITRvQKpMdlj8sSqiGAmFsPLsCOyN3vu%2BWmNtPkLRRZPOx2ALeWiEZr
e56EZVr70q%2FZHEZZ7spEW51KObxqddzsRv3wKTyNYJBeb85%2FWRK9Fq2OTCyCKYr3F7aKN3QBmmXcZ824hYjj6aSbAN%2BGEuwWd9Oxtr6iJHWCE
gAmJXXENFVFMfw%2FdxWkHFaXuHH4YYKacL9Q0fbwifnsRd%2BRAlc8yoR07Q4VeUQF3LJHvpljvXyBpICQPhkV2a2alQP5h6yMbdrmu409O

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed1f-ae48-70ad-bf1c-272dd3f95091
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed22-61e2-7b31-bea8-790ce8b4b638
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed28-2628-74ed-ac9c-106349b709d4
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed29-8255-7ef1-b0c9-cc6f4b05bbc7
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed2a-94ae-7531-b8bb-f95b9315b991


In [57]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-e2322b567f5b478ab00e0d42861fa885',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [ ]:
from openai.types.chat.chat_completion_message_function_tool_call import ChatCompletionMessageFunctionToolCall

def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[ChatCompletionMessageFunctionToolCall]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        with console.status(f"[bold cyan]Executing {function_name}...[/bold cyan]"):
            tool = get_tool(tools, function_name)
            tool_response = tool(**function_args)
        
        # Create panel content
        panel_content = f"[bold cyan]🔧 Tool Call:[/bold cyan] {function_name}\n\n"
        panel_content += f"[dim]Args: {tool_call.function.arguments}[/dim]\n\n"
        
        if isinstance(tool_response, list):
            panel_content += f"[green]✓[/green] Found {len(tool_response)} results"
        else:
            panel_content += f"[green]✓[/green] {function_name} executed successfully"
        
        console.print(Panel(panel_content, border_style="cyan"))
        
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [81]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
console.md(final_response.content)


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "how to cook seabass step by step", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

<tool_call> {"name": "exa_search", "arguments": {"query": "how to cook seabass step by step", "num_results": 5}}   
</tool_call>

Let's wrap this in a function:

In [82]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL, 
        messages=messages, 
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
    
    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [83]:
result = research("What are the most popular pokemons?")
console.md(result)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "most popular Pokémon of all time", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Based on the search results, the most popular Pokémon vary slightly depending on the source and criteria (e.g., fan
votes, cultural impact, merchandise, or TCG prices), but several names consistently rise to the top.               

According to a large 2019 survey of over 52,000 Pokémon fans on Reddit, the top 5 most popular Pokémon were:       

 1 Charizard (Fire/Flying-type): The iconic final evolution of Charmander, known for its powerful design and status
   as a fan favorite, even surpassing Pikachu in this poll.                                                        
 2 Gengar (Ghost/Poison-type): A beloved ghost-type Pokémon famous for its mischievous personality and cool factor.
 3 Arcanine (Fire-type): A majestic, dog-like Pokémon often praised for its design and power.                      
 4 Bulbasaur (Grass/Poison-type): The first Pokémon in the Pokédex and a starter choice, which has maintained      
   immense popularity.                                                                                             
 5 Blaziken (Fire/Fighting-type): The final evolution of the Hoenn starter Torchic, known for its aggressive style 
   and powerful attacks.                                                                                           

Other sources highlight different popular Pokémon based on different metrics:                                      

 • Pikachu, while ranked 44th in the Reddit survey, is consistently mentioned as the most iconic and recognizable  
   face of the entire franchise due to its role as Ash's companion and the series mascot.                          
 • Eevee and its evolutions (like Umbreon and Vaporeon) are frequently cited as fan favorites due to their cuteness
   and versatility.                                                                                                
 • Mewtwo is noted for its powerful design and role in the first Pokémon movie.                                    
 • Lists also often include Lucario, Snorlax, Greninja, and Gyarados as highly popular choices.                    

In summary, while Charizard topped a major fan vote, the "most popular" Pokémon can depend on context, with Pikachu
being the most iconic symbol of the brand, and a group including Gengar, Arcanine, Bulbasaur, Eevee, and Lucario   
forming a core of consistently beloved fan favorites.                                                              

Sources:                                                                                                           

 • The 25 Most Popular Pokémon, Based on More Than 52,000 Fan ... (Business Insider)                               
 • The Most Popular Pokémon: A Look at Fan Favorites (Merchandise4All)                                             
 • 10 Most Popular Pokemon of All Time (Outrun Gaming)

![](images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [91]:
@weave.op
def research_loop(query: str, max_turns: int = 4, tools = [exa_search, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format."},
        {"role": "user", "content": query}]
    
    for turn in range(max_turns):
        console.rule(f"Agent Loop Turn {turn + 1}/{max_turns}")

        # call model with tools
        response = call_model(
            model_name=MODEL_MEDIUM, 
            messages=messages, 
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=[exa_search, add_numbers], tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            console.rule("Final Model Response")
            console.md(response.content)
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [92]:
res = research_loop("What is the sum of the populations of the 2 major EU cities?")

─────────────────────────────────────────────── Agent Loop Turn 1/4 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "largest cities in the European Union by population 2024", "num_results": 5}                    │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/4 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 3897145, "b": 3460771}                                                                              │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 3/4 ───────────────────────────────────────────────

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Answer: 7,357,916                                                                                                  

Based on the most recent population data for cities within the European Union, the two largest cities by population
within city proper limits are:                                                                                     

 • Berlin, Germany – 3,897,145                                                                                     
 • Madrid, Spain – 3,460,771                                                                                       

The sum of these two populations is:                                                                               

[ 3,897,145 + 3,460,771 = 7,357,916 ]

# 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [93]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [ ]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages, 
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=self.tools, tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                messages.append(response.model_dump())
                final_assistant_content = response.content
        except Exception as e:
            console.print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return AgentState(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        state = AgentState(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}])
        for _ in range(max_turns):
            console.rule(f"Agent Loop Turn {state.step+1}/{max_turns}")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state


![](images/07_simple_agent.png)

In [99]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


────────────────────────────────────────────── Agent Loop Turn 0/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash's first 3 Pokémon names and their weights", "num_results": 5}                              │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash's first 3 Pokémon names and their weights", "num_results": 5}                              │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 2/10 ───────────────────────────────────────────────

ERROR in Agent Step: Error code: 400 - {'error': {'message': "This model's maximum context length is 262144 tokens.
However, your request has 297207 input tokens. Please reduce the length of the input messages. None", 'type': 
'BadRequestError', 'param': None, 'code': 400}}

Final response: Agent error in step 3: Error code: 400 - {'error': {'message': "This model's maximum context length is 262144 tokens. However, your request has 297207 input tokens. Please reduce the length of the input messages. None", 'type': 'BadRequestError', 'param': None, 'code': 400}}


In [104]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search_and_refine(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    @weave.op
    def refine_search_result(result, query):
        messages = [
            {"role":"system", "content": f"Your task is to extract from the search results only the info that is relevant to answer the query"},
            {"role": "user", "content": f"- query: {query}\n- Search result: {result}"}
        ]
        refined_search = call_model(model_name=MODEL_SMALL, messages=messages)
        return refined_search.content

    output = []
    for result in search_results.results:
        refined_text = refine_search_result(result.text, query)
        output.append(
            {"title": result.title,
            "text": refined_text,
            "url": result.url
            }
        )
    return output

In [105]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")

────────────────────────────────────────────── Agent Loop Turn 0/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search_and_refine                                                                             │
│                                                                                                                 │
│ Args: {"query": "Ash Ketchum first three Pokémon names and weights", "num_results": 5}                          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 1/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 6.0, "b": 2.9}                                                                                      │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 2/10 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 8.9, "b": 30.0}                                                                                     │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────── Agent Loop Turn 3/10 ───────────────────────────────────────────────

Final response: Ash's first three Pokémon are **Pikachu**, **Caterpie**, and **Pidgeotto**. Their combined weight is:

- **Pikachu**: 6.0 kg  
- **Caterpie**: 2.9 kg  
- **Pidgeotto**: 30.0 kg  

**Total weight**: 6.0 + 2.9 + 30.0 = **38.9 kg**

*Note: While Pikachu was given to Ash at the beginning, Caterpie was the first Pokémon he caught in the wild, and Pidgeotto was the third Pokémon he obtained.*

Sources:  
- [Game Rant - Ash's First 10 Pokémon, Ranked](https://gamerant.com/pokemon-ashs-first-pokemon-ranked/)  
- [Bulbapedia - Ash Ketchum](https://bulbapedia.bulbagarden.net/wiki/Ash_Ketchum)


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.